<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.np.edu.sg/PublishingImages/Pages/default/odp/ICT.jpg' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Deep Learning</h1><h2>Practical 9a - Keras Functional API</h2><h3>AY2020/21 Semester</h3></th>
    </tr>
</table>

### 1. Introduction to the functional API
In the functional API, you directly manipulate tensors, and you use layers as functions
that take tensors and return tensors (hence, the name functional API):

In [1]:
from tensorflow.keras import Input, layers
input_tensor = Input(shape=(32,))

In [2]:
dense = layers.Dense(32, activation='relu')
output_tensor = dense(input_tensor)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Let’s start with a minimal example that shows side by side a simple Sequential model
and its equivalent in the functional API:


In [3]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

* ***A Sequential Model***

In [4]:
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


* ***A Functional API Model***

In [5]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


The only part that may seem a bit magical at this point is instantiating a `Model` object
using only an input tensor and an output tensor. Behind the scenes, Keras retrieves
every layer involved in going from `input_tensor` to `output_tensor`, bringing them
together into a graph-like data structure—a `Model`. Of course, the reason it works is
that `output_tensor` was obtained by repeatedly transforming `input_tensor`.

When it comes to compiling, training, or evaluating such an instance of `Model`, the
API is the same as that of Sequential:

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics =['acc'])
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs=10, batch_size=128)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 277us/sample - loss: 12.3934 - acc: 0.1090
Epoch 2/10
1000/1000 [==============================] - 0s 19us/sample - loss: 13.4638 - acc: 0.1110
Epoch 3/10
1000/1000 [==============================] - 0s 25us/sample - loss: 15.6799 - acc: 0.1140
Epoch 4/10
1000/1000 [==============================] - 0s 20us/sample - loss: 19.0378 - acc: 0.1130
Epoch 5/10
1000/1000 [==============================] - 0s 24us/sample - loss: 23.2789 - acc: 0.1130
Epoch 6/10
1000/1000 [==============================] - 0s 21us/sample - loss: 28.4827 - acc: 0.1130
Epoch 7/10
1000/1000 [==============================] - 0s 22us/sample - loss: 34.5078 - acc: 0.1160
Epoch 8/10
1000/1000 [==============================] - 0s 22us/sample - loss: 41.2618 - acc: 0.1160
Epoch 9/10
1000/1000 [==============================] - 0s 18us/sample - loss: 48.4857 - acc: 0.1120
Epoch 10/10
1000/1000 [==============================] - 0s 26us/sam

In [7]:
score = model.evaluate(x_train, y_train, verbose=2)

1000/1000 - 0s - loss: 61.7702 - acc: 0.1120


### 2. Build a multiple inputs model using functional API
We will build a question-answering model using functional API. The model has two inputs: a natural-language question
and a text snippet (such as a news article) providing information to be used for answering the question. The model merge two input branches using a `concatenate` layer, after that model produce an answer via a softmax over some predefined vocabulary.

First, we build up model using funcational API.

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(100,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(100,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],axis=-1)
answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 64)      640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 32)      320000      question

Then, we generate some dummy inputs (text, questions, answers) using `numpy` random number generation fuction.

In [9]:
import numpy as np
num_samples = 1000
max_length = 100
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size=(num_samples, answer_vocabulary_size))

In [10]:
print(answers.shape)

(1000, 500)


Finally, we train the model using the above dummy inputs.

In [11]:
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 775554.2945 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 777532.6000 - acc: 0.0020
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 779928.7930 - acc: 0.0010
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 781340.7695 - acc: 0.0020
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 782832.3425 - acc: 0.0020
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 784261.0315 - acc: 0.0020
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 785121.6370 - acc: 0.0020
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 785563.6485 - acc: 0.0020
Epoch 9/10
1000/1000 [==============================

In [14]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes = True, to_file='model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
